# **Resumo da Análise Exploratória - Parte 02**
A intenção inicial era utilizar o modelo SVD e similares para gerar um modelo de recomendação de produtos, mas como visto na parte 01 da análise exploratória, não existe realmente um histórico de pedidos e avaliações feitas pelos usuários. Para dar continuidade nesse modelo de recomendação, vamos deixar de lado a recomendação com base no histórico de pedidos e avaliações, e seguiremos com um modelo de recomendação com base em características existentes nas bases de dados.

In [1]:
#Bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("session").getOrCreate()
spark.conf.set("spark.sql.adaptive.enable", "true")

In [15]:
#Datasets
olist_customers = spark.read.option("header", True).parquet("olist_customers_dataset.parquet") \
    .drop("customer_zip_code_prefix")
olist_orders = spark.read.option("header", True).parquet("olist_orders_dataset.parquet") \
    .select('order_id', 'customer_id', 'order_status')
olist_order_reviews = spark.read.option("header", True).parquet("olist_order_reviews_dataset.parquet") \
    .select('order_id', 'review_score')
olist_order_items = spark.read.option("header", True).parquet("olist_order_items_dataset.parquet") \
    .select('order_id', 'product_id', 'price', 'freight_value')
olist_products = spark.read.option("header", True).parquet("olist_products_dataset.parquet")

Agora vou fazer alguns filtros que foram feitos na parte 01 para ser reutilizado aqui, como filtrar pedidos que um único tipo de item, com status de entregue e que foram avalidados. Assim amenizando o volume do cruazmento dos dados.

In [16]:
#Lista de produtos únicos nos pedidos
produtos = olist_order_items.groupBy("order_id").agg(F.count("product_id").alias("total"),
                                                     F.countDistinct("product_id").alias("distinto"))
lista_de_pedidos_com_unico_produto = produtos.filter(F.col("distinto") == 1).select("order_id")

In [17]:
#Filtrando os order_id que estão dentro da lista de pedidos com único tipo de produto
olist_orders = olist_orders.join(lista_de_pedidos_com_unico_produto, on="order_id", how="inner")

#Filtrando os order_id que estão dentro da lista de produtos com único tipo de produto
olist_order_items = olist_order_items.join(lista_de_pedidos_com_unico_produto, on="order_id", how="inner")
olist_order_items = olist_order_items.dropDuplicates(["order_id"])

#Filtro de status de pedido como entregue e tirando essa coluna
olist_orders = olist_orders.filter(F.col("order_status") == "delivered")
olist_orders = olist_orders.drop("order_status")

#Filtrando os pedidos que estão sem avaliações e com order_id duplicados
olist_order_reviews = olist_order_reviews.dropna(subset=["review_score"], how="any")
olist_order_reviews = olist_order_reviews.dropDuplicates(["order_id"])

In [18]:
#Cruzamentos
dataset = olist_orders.join(olist_customers, on="customer_id", how="inner") \
    .join(olist_order_items, on="order_id", how="inner") \
    .join(olist_products, on="product_id", how="inner") \
    .join(olist_order_reviews, on="order_id", how="inner")

In [19]:
#Comparando a qtd de linhas da base nova e da base de pedidos
dataset.count(), olist_orders.count(), olist_orders.count() - dataset.count()

(92673, 93281, 608)

In [25]:
# Lista para armazenar os resultados
null_counts = []

for c in dataset.columns:
    count_nulls = dataset.filter(F.col(c).isNull()).count()
    null_counts.append((c, count_nulls))

# Transformar em DataFrame
schema = ["variavel", "qtd_nulos"]
nulls_df = spark.createDataFrame(null_counts, schema)

# Mostrar o resultado
nulls_df.orderBy(F.desc("qtd_nulos")).show(truncate=False)

+--------------------------+---------+
|variavel                  |qtd_nulos|
+--------------------------+---------+
|product_category_name     |1317     |
|product_name_lenght       |1317     |
|product_description_lenght|1317     |
|product_photos_qty        |1317     |
|product_weight_g          |16       |
|product_height_cm         |16       |
|product_length_cm         |16       |
|product_width_cm          |16       |
|price                     |0        |
|order_id                  |0        |
|freight_value             |0        |
|review_score              |0        |
|customer_id               |0        |
|product_id                |0        |
|customer_unique_id        |0        |
|customer_city             |0        |
|customer_state            |0        |
+--------------------------+---------+



In [26]:
#Base para exportação
dataset.write.parquet("dataset.parquet")